In [ ]:
from __future__ import division, print_function

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel,WhiteKernel
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import BicScore
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.models import BayesianModel

import shapely.wkt as shpwkt

%pylab inline

In [ ]:
# yelp_cluster.to_csv(DATA + '/Yelp_Weights_subset.csv')
yelp_cluster = pd.read_csv('../data/Yelp_Weights_subset.csv')
yelp_cluster.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace=True)
yelp_cluster.head()

In [ ]:
df = gpd.GeoDataFrame(yelp_cluster)
df['geometry'] = df['geometry'].apply(lambda x: shpwkt.loads(x))
df = gpd.GeoDataFrame(yelp_cluster, geometry='geometry')
df.crs = {'init': 'epsg:4326'}
# df.crs = {'init': 'epsg:4326'} 
df.head()

In [ ]:
X = df.iloc[:,8:41]

In [ ]:
range_n_clusters = range(2,10)
for n in range_n_clusters:
    #19900
    gmm = GaussianMixture(n_components=n,random_state=3391)
    res = gmm.fit(X)
    silhouette_avg = silhouette_score(X, res.predict(X))
    print("For n_clusters = {},".format(n)+" the average silhouette_score is : {}".format(silhouette_avg))

In [ ]:
0.240221382689

In [ ]:
fig = plt.figure(figsize=(15,24))
for n in range(2,8):
    X = yelp_cluster.iloc[:,8:41]
    km = KMeans(random_state=3391, n_clusters=n)
    res = km.fit(X)

    yelp_cluster['label'] = res.labels_
    X['label'] = res.labels_
    
    ax = fig.add_subplot(319+n)
    yelp_cluster.plot(ax=ax, column='label', cmap='viridis', legend=True, categorical=True, edgecolor='lightgray')

In [ ]:
X = yelp_cluster.iloc[:,8:41]
X.head(1)
km = KMeans(random_state=3391, n_clusters=7)
res = km.fit(X)

df['label'] = res.labels_
X['label'] = res.labels_

# fig = plt.figure(figsize=(15,12))
# ax = fig.add_subplot(111)
# df.plot(ax=ax, column='label', cmap='Reds', legend=True, categorical=True, edgecolor='lightgray')

In [ ]:
df.plot()

In [ ]:
dfs = []
for i in range(7):
    dfs.append(X[X['label'] == i].mean())
label_df = pd.concat(dfs, axis=1)
# label_df.columns = ['Domin/Latin', 'Manhattan', 'Cari/S.Asia', 'Cari/Black', 'UPPER/DTBL', 'Asian', 'Jewish']
label_df

In [ ]:
yelp_cluster.head(1)

In [ ]:
y = yelp_cluster[['FIPS','label']]

In [ ]:
import pandas as pd

In [ ]:
DATA = '../data'

In [ ]:
ACS = pd.read_csv(DATA + '/features/ACSselectedData.csv', index_col=0)
Health = pd.read_csv(DATA + '/features/NYC_Health_Features.csv')
# Yelp = pd.read_csv(DATA + '/yelp_counts_per_ct.csv', index_col=0)

In [ ]:
MergedData = ACS.merge(Health, left_on='FIPS', right_on='TractFIPS', how='outer')
print(MergedData.shape)
MergedData.head()

In [ ]:
list(MergedData.columns)

In [ ]:
MergedData = MergedData[['FIPS',
 'Census Tract',
 'Pop Density',
 'Income',
 'Age',
 'Household Size',
 'Total Pop',
 'DIABETES_CrudePrev',
 'HIGHCHOL_CrudePrev',
 'OBESITY_CrudePrev']]

In [ ]:
MergedData.shape

In [ ]:
MergedData.head(1)

In [ ]:
MergedData = MergedData.merge(y, on='FIPS', how='outer')
print(MergedData.shape)
MergedData.head()

In [ ]:
MergedData.dropna(axis=0,how='any',inplace=True)

In [ ]:
MergedData.head(1)

In [ ]:
MergedData.shape

In [ ]:
MergedData

In [ ]:
X = MergedData.iloc[:,2:]

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('dataformodel.csv', index_col=0)
df.head(1)

,FIPS,Census Tract,Pop Density,Income,Age,Household Size,Total Pop,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,OBESITY_CrudePrev,label
1,36005000200,200.0,28365.12,70893.0,38.6,3.94,5251.0,12.7,33.8,29.9,6.0


In [41]:
X = df.iloc[:,2:]

In [42]:
for col in X.columns[:-1]:
    X[col] = pd.cut(X[col],7,labels=[0,1,2,3,4,5,6])
X.head()

,Pop Density,Income,Age,Household Size,Total Pop,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,OBESITY_CrudePrev,label
1,0,1,2,3,1,2,3,4,6.0
2,0,1,2,2,1,2,2,3,3.0
3,0,0,2,2,1,3,3,5,6.0
4,0,0,2,1,0,2,2,4,6.0
5,1,0,2,2,2,3,3,5,3.0


# HillClimbSearch + BdeuScore, K2Score, BicScore

In [17]:
# your answers here
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import BdeuScore, K2Score, BicScore
import numpy as np

In [15]:
np.random.seed(300)

In [16]:
hc = HillClimbSearch(X, scoring_method=BicScore(X))
best_model = hc.estimate()
print(best_model.edges())

[('Pop Density', 'Total Pop'), ('OBESITY_CrudePrev', 'Age'), ('OBESITY_CrudePrev', 'label'), ('DIABETES_CrudePrev', 'HIGHCHOL_CrudePrev'), ('DIABETES_CrudePrev', 'OBESITY_CrudePrev'), ('Income', 'Pop Density'), ('Income', 'DIABETES_CrudePrev'), ('label', 'Household Size')]


In [18]:
hc = HillClimbSearch(X, scoring_method=BdeuScore(X))
best_model = hc.estimate()
print(best_model.edges())

[('Pop Density', 'Total Pop'), ('Pop Density', 'Income'), ('HIGHCHOL_CrudePrev', 'Age'), ('OBESITY_CrudePrev', 'HIGHCHOL_CrudePrev'), ('OBESITY_CrudePrev', 'label'), ('DIABETES_CrudePrev', 'HIGHCHOL_CrudePrev'), ('DIABETES_CrudePrev', 'OBESITY_CrudePrev'), ('Income', 'Age'), ('Income', 'DIABETES_CrudePrev'), ('label', 'Household Size')]


In [19]:
hc = HillClimbSearch(X, scoring_method=K2Score(X))
best_model = hc.estimate()
print(best_model.edges())

[('Pop Density', 'Total Pop'), ('HIGHCHOL_CrudePrev', 'Age'), ('HIGHCHOL_CrudePrev', 'label'), ('HIGHCHOL_CrudePrev', 'OBESITY_CrudePrev'), ('OBESITY_CrudePrev', 'label'), ('DIABETES_CrudePrev', 'label'), ('DIABETES_CrudePrev', 'Income'), ('DIABETES_CrudePrev', 'HIGHCHOL_CrudePrev'), ('DIABETES_CrudePrev', 'OBESITY_CrudePrev'), ('Income', 'Age'), ('Income', 'Pop Density'), ('label', 'Household Size')]


# ConstraintBasedEstimator

In [20]:
from pgmpy.estimators import ConstraintBasedEstimator

In [24]:
X.columns

Index([u'Pop Density', u'Income', u'Age', u'Household Size', u'Total Pop',
       u'DIABETES_CrudePrev', u'HIGHCHOL_CrudePrev', u'OBESITY_CrudePrev',
       u'label'],
      dtype='object')

In [46]:
est = ConstraintBasedEstimator(X)

In [36]:
skel, separating_sets = est.estimate_skeleton(significance_level=0.01)
print("Undirected edges: ", skel.edges())

('Undirected edges: ', [('Income', 'label'), ('Income', 'OBESITY_CrudePrev'), ('Pop Density', 'label'), ('Pop Density', 'OBESITY_CrudePrev'), ('label', 'OBESITY_CrudePrev')])


In [37]:
pdag = est.skeleton_to_pdag(skel, separating_sets)
print("PDAG edges:       ", pdag.edges())

('PDAG edges:       ', [('OBESITY_CrudePrev', 'Pop Density'), ('OBESITY_CrudePrev', 'label'), ('OBESITY_CrudePrev', 'Income'), ('Pop Density', 'OBESITY_CrudePrev'), ('Pop Density', 'label'), ('label', 'Pop Density'), ('label', 'OBESITY_CrudePrev'), ('label', 'Income'), ('Income', 'OBESITY_CrudePrev'), ('Income', 'label')])


In [38]:
model = est.pdag_to_dag(pdag)
print("DAG edges:        ", model.edges())

('DAG edges:        ', [('OBESITY_CrudePrev', 'Pop Density'), ('OBESITY_CrudePrev', 'label'), ('OBESITY_CrudePrev', 'Income'), ('label', 'Pop Density'), ('label', 'Income')])


# ExhaustiveSearch

In [45]:
X1 = X[[u'Pop Density', u'Income', 
     u'OBESITY_CrudePrev',
       u'label']]

In [48]:
from pgmpy.estimators import ExhaustiveSearch

bic = BicScore(X1)
# Note: exhaustive search will be terribly expensive for more than a few variables
es = ExhaustiveSearch(X1, scoring_method=bic)
best_model = es.estimate()
print(best_model.edges())

print("\nAll DAGs by score:")
for score, dag in reversed(es.all_scores()):
    print(score, dag.edges())

[('OBESITY_CrudePrev', 'label'), ('Income', 'Pop Density'), ('Income', 'OBESITY_CrudePrev')]

All DAGs by score:
(-11609.812916944364, [('OBESITY_CrudePrev', 'label'), ('Income', 'Pop Density'), ('Income', 'OBESITY_CrudePrev')])
(-11609.812916944365, [('OBESITY_CrudePrev', 'Income'), ('label', 'OBESITY_CrudePrev'), ('Income', 'Pop Density')])
(-11609.812916944365, [('OBESITY_CrudePrev', 'Income'), ('OBESITY_CrudePrev', 'label'), ('Income', 'Pop Density')])
(-11609.812916944365, [('Pop Density', 'Income'), ('OBESITY_CrudePrev', 'label'), ('Income', 'OBESITY_CrudePrev')])
(-11675.395582057841, [('OBESITY_CrudePrev', 'label'), ('label', 'Pop Density'), ('Income', 'OBESITY_CrudePrev')])
(-11675.395582057843, [('OBESITY_CrudePrev', 'Income'), ('label', 'Pop Density'), ('label', 'OBESITY_CrudePrev')])
(-11675.395582057843, [('Pop Density', 'label'), ('OBESITY_CrudePrev', 'Income'), ('label', 'OBESITY_CrudePrev')])
(-11675.395582057843, [('OBESITY_CrudePrev', 'Income'), ('OBESITY_CrudePrev', 

In [52]:
X2 = X[[u'Pop Density', u'Income', u'Household Size',
     u'OBESITY_CrudePrev',
       u'label']]

In [53]:
bic = BicScore(X2)
# Note: exhaustive search will be terribly expensive for more than a few variables
es = ExhaustiveSearch(X2, scoring_method=bic)
best_model = es.estimate()
print(best_model.edges())

print("\nAll DAGs by score:")
i = 0
for score, dag in reversed(es.all_scores()):
    if i < 5:
        print(score, dag.edges())
        i += 1
    else:
        break

[('Household Size', 'label'), ('OBESITY_CrudePrev', 'Income'), ('label', 'OBESITY_CrudePrev'), ('Income', 'Pop Density')]

All DAGs by score:
(-13857.904369294631, [('Household Size', 'label'), ('OBESITY_CrudePrev', 'Income'), ('label', 'OBESITY_CrudePrev'), ('Income', 'Pop Density')])
(-13857.904369294632, [('OBESITY_CrudePrev', 'Income'), ('label', 'Household Size'), ('label', 'OBESITY_CrudePrev'), ('Income', 'Pop Density')])
(-13857.904369294632, [('OBESITY_CrudePrev', 'Income'), ('OBESITY_CrudePrev', 'label'), ('label', 'Household Size'), ('Income', 'Pop Density')])
(-13857.904369294632, [('Pop Density', 'Income'), ('OBESITY_CrudePrev', 'label'), ('label', 'Household Size'), ('Income', 'OBESITY_CrudePrev')])
(-13857.904369294632, [('OBESITY_CrudePrev', 'label'), ('label', 'Household Size'), ('Income', 'Pop Density'), ('Income', 'OBESITY_CrudePrev')])
